In [1]:
from sknn import mlp
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer,LabelBinarizer
from sklearn.cross_validation import train_test_split
%matplotlib inline


Couldn't import dot_parser, loading of dot files will not be possible.


Using gpu device 0: GeForce 920M (CNMeM is disabled, cuDNN not available)
/home/mrafi123/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [1]:
import theano

Couldn't import dot_parser, loading of dot files will not be possible.


Using gpu device 0: GeForce 920M (CNMeM is disabled, cuDNN not available)


In [2]:
X_train=np.load('/media/mrafi123/UStore/Dimensionality-Reduction/data/CIFAR10/X_train.npy')
X_test=np.load('/media/mrafi123/UStore/Dimensionality-Reduction/data/CIFAR10/X_test.npy')
y_train=np.load('/media/mrafi123/UStore/Dimensionality-Reduction/data/CIFAR10/y_train.npy')
y_test=np.load('/media/mrafi123/UStore/Dimensionality-Reduction/data/CIFAR10/y_test.npy')

In [3]:
print (X_train.shape)
print (y_train.shape)
print (X_test.shape)
print (y_test.shape)

(50000, 32, 32, 3)
(50000,)
(10000, 32, 32, 3)
(10000,)


In [4]:
x_train = X_train.astype('float32') / 255.
x_test = X_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
print (x_train.shape)
print (x_test.shape)

(50000, 3072)
(10000, 3072)


In [5]:
n_feat = x_train.shape[1]
n_targets = y_train.max() + 1
print(n_feat)
print(n_targets)

3072
10


In [6]:
errors = []
def store_stats(avg_valid_error, avg_train_error, **_):
    errors.append((avg_valid_error, avg_train_error))
    print ('Average Validation Error ',avg_valid_error)
    print ('Average Training Error ',avg_train_error)


In [7]:
#be careful of the humungous verbose
def my_callback(event, **variables):
    print(event)        # The name of the event, as shown in the list above.
    print(variables)    # Full dictionary of local variables from training loop.


In [52]:
#be careful of the humungous verbose
def my_callback(**variables):
    #print(variables)    # Full dictionary of local variables from training loop.
    print('best_params',len(variables['best_params']))
    print('best_params 0',len(variables['best_params'][0]))
    print('best_params 0 0',len(variables['best_params'][0][0]))
    print('best_params 0 1',len(variables['best_params'][0][1]))
    print('best_params 1',len(variables['best_params'][1]))
    print('best_params 1 0',len(variables['best_params'][1][0]))
    print('best_params 1 1',len(variables['best_params'][1][1]))



In [8]:
def my_callback(**variables):
#def my_callback(event, **variables):
    #print(event)        # The name of the event, as shown in the list above.
    print(variables['y'])    # Full dictionary of local variables from training loop.
    print(variables['yb'])
    expected=variables['yb']
    predicted = variables['y']
    num=len(expected)
    r=0
    w=0
    for i in range(num):
        #print ('y_pred ',y_pred[i])
        #print ('labels ',labels[i])
        #without the use of all() returns error truth value of an array with more than one element is ambiguous
        #if y_pred[i].all() == labels[i].all():
        if np.array_equal(expected[i],predicted[i]):
            r+=1
        else:
            w+=1
    print ("Trained ",  num, "Cifar 10 images")
    print ("correct: ", r, "wrong: ", w, "Training error%: ", float(w)*100/(r+w), "%")
    print ("Training Accuracy ", float(r)*100/(r+w), "%")

In [9]:
print (x_train[49000:].shape)
print (y_train[49000:].shape)
print (x_test[9000:].shape)
print (y_test[9000:].shape)

sample_train = x_train[49000:]
sample_test = x_test[9000:]
sample_y_train = y_train[49000:]
sample_y_test = y_test[9000:]
print (sample_train.shape)
print (sample_test.shape)
print (sample_y_train.shape)
print (sample_y_test.shape)


(1000, 3072)
(1000,)
(1000, 3072)
(1000,)
(1000, 3072)
(1000, 3072)
(1000,)
(1000,)


In [10]:
n_feat = sample_train.shape[1]
n_targets = sample_y_train.max() + 1
print(n_feat)
print(n_targets)

3072
10


In [53]:
nn = mlp.Classifier(
        layers=[
            mlp.Layer("Sigmoid", units=n_feat/16),
            mlp.Layer("Softmax", units=n_targets)],
        n_iter=1,
        n_stable=10,
        batch_size=250,
        learning_rate=0.002,
        learning_rule="momentum",
        valid_size=0.1,
        verbose=1,
        callback={'on_epoch_finish': my_callback}
)

In [54]:
nn.fit(sample_train, sample_y_train)


/home/mrafi123/anaconda3/lib/python3.5/site-packages/lasagne/init.py:99: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  low=self.range[0], high=self.range[1], size=shape))
/home/mrafi123/anaconda3/lib/python3.5/site-packages/numpy/core/numeric.py:190: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  a = empty(shape, dtype, order)


best_params 2
best_params 0 2
best_params 0 0 3072
best_params 0 1 192
best_params 1 2
best_params 1 0 192
best_params 1 1 10


Classifier(batch_size=250,
      callback={'on_epoch_finish': <function my_callback at 0x7f46a614e9d8>},
      debug=False, dropout_rate=None, f_stable=0.001,
      hidden0=<sknn.nn.Layer `Sigmoid`: frozen=False, units=192.0, name='hidden0'>,
      layers=[<sknn.nn.Layer `Sigmoid`: frozen=False, units=192.0, name='hidden0'>, <sknn.nn.Layer `Softmax`: frozen=False, units=10, name='output'>],
      learning_momentum=0.9, learning_rate=0.002, learning_rule='momentum',
      loss_type=None, n_iter=1, n_stable=10, normalize=None,
      output=<sknn.nn.Layer `Softmax`: frozen=False, units=10, name='output'>,
      parameters=None, random_state=None, regularize=None,
      valid_set=(array([[ 0.61961,  0.55294, ...,  1.     ,  1.     ],
       [ 0.71373,  0.76863, ...,  0.61569,  0.58039],
       ...,
       [ 0.92941,  0.92941, ...,  0.64706,  0.71373],
       [ 0.18824,  0.3098 , ...,  0.23137,  0.27451]], dtype=float32), array([[ 0.,  0., ...,  0.,  0.],
       [ 0.,  0., ...,  1.,  0.],
 

In [10]:
nn = mlp.Classifier(
        layers=[
            mlp.Layer("Tanh", units=n_feat/8),
            mlp.Layer("Sigmoid", units=n_feat/16),
            mlp.Layer("Softmax", units=n_targets)],
        n_iter=50,
        n_stable=10,
        batch_size=25,
        learning_rate=0.002,
        learning_rule="momentum",
        valid_size=0.1,
        verbose=1,
        callback={'on_epoch_finish': store_stats}
)

In [ ]:
nn.fit(x_train, y_train)


In [11]:


from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

expected = y_test
predicted = nn.predict(x_test)

print("Classification report for classifier %s:\n%s\n" % (
    nn, classification_report(expected, predicted)))
print("Confusion matrix:\n%s" % confusion_matrix(expected, predicted))

/home/mrafi123/anaconda3/lib/python3.5/site-packages/lasagne/init.py:99: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  low=self.range[0], high=self.range[1], size=shape))
/home/mrafi123/anaconda3/lib/python3.5/site-packages/numpy/core/numeric.py:190: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  a = empty(shape, dtype, order)


Average Validation Error  1.83768300056
Average Training Error  1.94756979227
Average Validation Error  1.7366314137
Average Training Error  1.77474173817
Average Validation Error  1.69711952567
Average Training Error  1.69599910902
Average Validation Error  1.6377687192
Average Training Error  1.63820813225
Average Validation Error  1.60753723919
Average Training Error  1.58994910942
Average Validation Error  1.57074083149
Average Training Error  1.5500730788
Average Validation Error  1.56890162289
Average Training Error  1.51622480018
Average Validation Error  1.56587384909
Average Training Error  1.48971211003
Average Validation Error  1.51451502413
Average Training Error  1.45850133257
Average Validation Error  1.49152029604
Average Training Error  1.4364023799
Average Validation Error  1.45823601097
Average Training Error  1.40679898318
Average Validation Error  1.4288492018
Average Training Error  1.38514340722
Average Validation Error  1.46831718445
Average Training Error  1.361

In [23]:
errors

[(1.8376830005645752, 1.9475697922706603),
 (1.7366314136981964, 1.7747417381736967),
 (1.6971195256710052, 1.6959991090165245),
 (1.6377687191963195, 1.6382081322537527),
 (1.6075372391939162, 1.5899491094218361),
 (1.5707408314943314, 1.550073078804546),
 (1.5689016228914261, 1.5162248001827134),
 (1.5658738490939141, 1.4897121100293265),
 (1.514515024125576, 1.4585013325677978),
 (1.4915202960371972, 1.4364023799035284),
 (1.4582360109686852, 1.406798983183172),
 (1.4288492017984391, 1.3851434072189861),
 (1.4683171844482421, 1.3616377874215444),
 (1.4255577343702317, 1.3407566630840302),
 (1.4302572271227836, 1.3216965245869425),
 (1.4011410796642303, 1.3031028749876552),
 (1.3732928878068924, 1.2805527448654175),
 (1.3728320482373237, 1.2662964458929167),
 (1.423305741250515, 1.249551374150647),
 (1.4072544631361961, 1.23097586641709),
 (1.3921086618304253, 1.2163592107428445),
 (1.3765971279144287, 1.1970941087603568),
 (1.3539567536115646, 1.1792999832828839),
 (1.40119317471981

In [21]:
expected.shape

(10000, 1)

In [20]:
expected=np.reshape(expected,(10000,1))

In [18]:
predicted.shape

(10000, 1)

In [19]:
predicted

array([[3],
       [8],
       [0],
       ..., 
       [3],
       [2],
       [7]])

In [22]:
num=len(expected)
r=0
w=0
for i in range(num):
        #print ('y_pred ',y_pred[i])
        #print ('labels ',labels[i])
        #without the use of all() returns error truth value of an array with more than one element is ambiguous
        #if y_pred[i].all() == labels[i].all():
        if np.array_equal(expected[i],predicted[i]):
            r+=1
        else:
            w+=1
print ("tested ",  num, "Cifar 10 images")
print ("correct: ", r, "wrong: ", w, "error rate: ", float(w)*100/(r+w), "%")
print ("got correctly ", float(r)*100/(r+w), "%")

tested  10000 Cifar 10 images
correct:  5318 wrong:  4682 error rate:  46.82 %
got correctly  53.18 %
